<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/snli_input_marge_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install pytorch_pretrained_bert
!pip install transformers
!git clone https://github.com/ronakdm/input-marginalization.git

In [3]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [4]:
import sys
sys.path.append("input-marginalization")
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from transformers import BertTokenizer, BertModel
from utils import generate_snli_dataloader
from models import LSTM
from torch.nn import LogSoftmax
import math
import torch.nn.functional as F

In [5]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
save_dir = "/content/gdrive/My Drive/NLP/imarg"

Mounted at /content/gdrive


In [6]:
SAMPLE_SIZE = 3
SIGMA = 1e-4
log_softmax = LogSoftmax(dim=0)

In [7]:
%%capture
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [8]:
lstm = torch.load(f"{save_dir}/lstm_snli.pt")

In [16]:
def loaddata():
  train_dataloader, validation_dataloader, test_dataloader = generate_snli_dataloader('input-marginalization/preprocessed_data/SNLI/snli_1.0', 1)
  return test_dataloader

In [10]:
def compute_probability(model, input_ids, attention_masks, label):
    logits = model(
        input_ids, token_type_ids=None, attention_mask=attention_masks, labels=label.repeat((len(input_ids))),
    ).logits
    
    return math.exp(logits[0][label])
    

In [11]:
def compute_probability2(model, input_ids, attention_masks, label):
  
    logits = model(
        input_ids.to(torch.int64), labels=None,
    )

    return torch.exp(torch.reshape(logits[:, label], (-1,)))

In [23]:
def calculate_woe(model, input_ids, attention_masks, label, sigma):
  device = "cuda" if next(model.parameters()).is_cuda else "cpu"
  bert_model.to(device)
  
  #predictions is the probability distribution of each word in the vocabulary for each word in input sentence
  predictions = bert_model(input_ids)
  print('OK')
  predictions = torch.squeeze(predictions)
  predictions = F.softmax(predictions, dim=1)

  #woe is the weight of evidence
  woe = []
  model.eval()

  print(predictions.shape)

  with torch.no_grad():
    for j in range (len(predictions)):
      word_scores = predictions[j]
      input_batch = input_ids.clone().to(device)
      
      #word_scores_batch calculates the value of the MLM of Bert for each masked word
      #we put 0 for the first input which is unmasked
      word_scores_batch = [0]

      for k in range(len(word_scores)):
        if word_scores[k] > sigma:
           input_batch = torch.cat((input_batch, input_ids), 0)
           input_batch[len(input_batch)-1][j] = k
           word_scores_batch.append(word_scores[k].item())
      
      #probability_input calculates the p(label|sentence) of the target model given each masked input sentence
      probability_input = compute_probability2(model, input_batch, attention_masks, label)
      
      m = torch.dot(torch.tensor(word_scores_batch).to(device), probability_input)
      logodds_input = math.log(probability_input[0] / (1-probability_input[0]))
      logodds_m = math.log(m / (1-m))
      woe.append(logodds_input-logodds_m)
  return woe


In [24]:
def input_marg(model): 
  test_data = loaddata()
  device = "cuda" if next(model.parameters()).is_cuda else "cpu"
  iter_data = iter(test_data)

  for i in range(SAMPLE_SIZE):
    nextsample = next(iter_data)
    sentences = nextsample[0]
    sentences = (sentences[0].to(device), sentences[1].to(device))
    labels =  nextsample[1].to(device)
    print("")
    #print(tokenizer.convert_ids_to_tokens(sentences))
    print(calculate_woe(model, sentences, None, labels, SIGMA))

In [25]:
input_marg(lstm)

TypeError: ignored

In [15]:
!ls

gdrive	input-marginalization  sample_data
